## Places Recommendation System using Foursquare API

# 1. Introduction 

We have to build recommender system which recommends locations based on previous ratings.
We know that tourist whenever travels to a location, he tries to find best spots around in that specific location to explore. They can't possibly try all the places hence this recommendation system helps them choose the best place around.

# 2. Data

### 2.1 Data Acquisition

1. To begin our analysis of Toronto neighborhoods, the first step is to obtain relevant data. We will retrieve this data from a Wikipedia page utilizing web scraping techniques, leveraging the `BeautifulSoup` library from Python.

    1.1 Once we have acquired the neighborhood data, our next task is to gather information about nearby venues using the Foursquare API.
    
    1.2 Additionally, we will fetch ratings for these venues using the Foursquare API.

2. In order to personalize our recommendations, we require some sample ratings from users. For this purpose, we will populate our dataset with demo values extracted from our earlier obtained dataframe.

This structured approach ensures a systematic and professional handling of data acquisition for our analysis.

### 2.2  Preprocessing

#### Importing libraries

In [197]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis from csv file
from bs4 import BeautifulSoup #web scraping
import requests # library to handle requests
import json # library to handle JSON files
from pandas import json_normalize # transform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import geopy.geocoders # convert an address into latitude and longitude values

import folium # map rendering library

print('Libraries are imported.')

Libraries are imported.


#### We are building recommender system for toronto as Chennai Dataset is unavailable for now, which can be scaled up later on for all the places

In [198]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# getting all the neighborhood data of Toronto
website_url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=prev&oldid=968782994').text
soup = BeautifulSoup(website_url, 'html.parser')

# Find the first table with the 'wikitable' class
my_table = soup.find('table', class_='wikitable')

# Verify that a table was found
if my_table:
    # Initialize an empty list to store the data
    data = []
    
    # Extract the table rows
    for tr in my_table.find_all('tr'):
        row = []
        for tds in tr.find_all('td'):
            # Clean the text and remove newline characters
            row.append(tds.text.strip())
        if row:
            data.append(row)

    # Create a DataFrame
    df = pd.DataFrame(data, columns=["PostalCode", "Borough", "Neighborhood"])
    
    # Print the first few rows of the DataFrame
    print(df.head())
else:
    print("Table not found")



  PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront


#### Writing our data into as .csv file for further use

In [199]:
# Assuming 'table1' is a comma-separated string of the table data
table1 = ",".join(["PostalCode", "Borough", "Neighborhood"]) + "\n"  # CSV headers
for row in data:
    table1 += ",".join(row) + "\n"  # Add the data rows

# Write the data to a CSV file
file_path = "./toronto.csv"
with open(file_path, "wb") as file:
    file.write(bytes(table1, encoding="ascii", errors="ignore"))


In [200]:
# Define the file path for toronto.csv here. 

#CHANGE THE LOCATION. RIGHT CLICK ON TORONTO.CSV, COPY RELATIVE PATH AND PASTE HERE 
file_path = "./toronto.csv"

# Use DataFrame.to_csv to write to a CSV file
df.to_csv(file_path, index=False)


#### Converting into dataframe and assigning columnnames

In [201]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]
df.head(10)

,Postalcode,Borough,Neighbourhood
0,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


### Data Cleaning


#### Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned. Droping row where borough is "Not assigned"

In [202]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough


In [203]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']

#### rows will be same postalcode will combined into one row with the neighborhoods separated with a comma

In [204]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()
df_new.head(15)

,Postalcode,Borough,Neighbourhood
0,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
6,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
7,M1B,Scarborough,"Malvern, Rouge"
8,M3B,North York,Don Mills
9,M4B,East York,"Parkview Hill, Woodbine Gardens"


In [205]:
df_new.shape

(104, 3)

#### We will be using a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data


In [206]:
import requests
import pandas as pd

url = 'http://cocl.us/Geospatial_data'
r = requests.get(url)
with open('Toronto_long_lat_data.csv', 'wb') as f:
    f.write(r.content)

df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
print(df_lon_lat.head())


  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [207]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

In [208]:
df_toronto = pd.merge(df_new,
                 df_lon_lat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
df_toronto.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Create a Map of Toronto City (with its Postal Codes' Regions)

In [209]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [210]:
import folium

# for the city Toronto, latitude and longitude are 
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# Assuming you have a dataframe 'df_toronto' with the columns 'Latitude', 'Longitude', 'Borough', 'Neighbourhood'
# Add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
# Display the map
map_toronto


# 3. Methodology

### Since our Api calls are limited we will focus on a particular borough for now

In [211]:
North_York_data = df_toronto[df_toronto['Borough'] == 'North York']
North_York_data = North_York_data.reset_index()
North_York_data.drop('index', axis=1, inplace=True)
North_York_data

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


#### Define Foursquare Credentials and Version


In [212]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = 20240321 # Foursquare API version (Today's date in YYYYMMDD)
# defining radius and limit of venues to get
radius=500
LIMIT=100

## Getting nearby venues 

In [213]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
# Part 1: creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
# Part 2: making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
# Part 3 returning only relevant information for each nearby venue and append to the list 
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    return (venues_list)

In [214]:
import requests
import pandas as pd

# Function to get nearby venues. Please fill in the placeholder values with your actual Foursquare credentials
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):

    CLIENT_ID = '' # Foursquare ID, replace with your Foursquare ID
    CLIENT_SECRET = '' # Foursquare Secret, replace with your Foursquare Secret
    ACCESS_TOKEN = '' # Foursquare Access Token for V3 API, replace with your access token
    VERSION = '20240321'  # Foursquare API version (Today's date in YYYYMMDD)
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        # Create API request URL
        url = f'https://api.foursquare.com/v3/places/search?ll={lat},{lng}&radius={radius}&limit={limit}'
        
        # Make the GET request
        headers = {
            "Accept": "application/json",
            "Authorization": f"{ACCESS_TOKEN}"
        }
        response = requests.get(url, headers=headers).json()
        
        # Append the relevant parts of the JSON to the venues_list
        for item in response['results']:
            venues_list.append({
                'Neighborhood': name,
                'Neighborhood Latitude': lat,
                'Neighborhood Longitude': lng,
                'Venue': item['name'],
                'Venue Latitude': item['geocodes']['main']['latitude'],
                'Venue Longitude': item['geocodes']['main']['longitude'],
                'Venue Category': item['categories'][0]['name'] if item['categories'] else ""
            })

    # Convert the venues_list to a DataFrame
    nearby_venues = pd.DataFrame(venues_list)
    return(nearby_venues)
print(North_York_data)
# Example usage:
# Replace 'North_York_data' with your actual DataFrame containing neighborhoods, latitude and longitude
# toronto_venues = getNearbyVenues(names=North_York_data['Neighbourhood'],
#                                   latitudes=North_York_data['Latitude'],
#                                   longitudes=North_York_data['Longitude'])

# print(toronto_venues.head())


   Postalcode     Borough                                    Neighbourhood  \
0         M3A  North York                                        Parkwoods   
1         M4A  North York                                 Victoria Village   
2         M6A  North York                 Lawrence Manor, Lawrence Heights   
3         M3B  North York                                        Don Mills   
4         M6B  North York                                        Glencairn   
5         M3C  North York                                        Don Mills   
6         M2H  North York                                Hillcrest Village   
7         M3H  North York  Bathurst Manor, Wilson Heights, Downsview North   
8         M2J  North York                     Fairview, Henry Farm, Oriole   
9         M3J  North York                  Northwood Park, York University   
10        M2K  North York                                  Bayview Village   
11        M3K  North York                                       

In [215]:
import requests

# Replace with your Foursquare credentials (avoid storing them in plain text)
API_KEY = ''
CLIENT_ID = ''
CLIENT_SECRET = ''
ACCESS_TOKEN = ''  #Check readme to get info on how to get access key as its not directly available
VERSION = '20240321'

# Replace with the latitude and longitude of your desired location
toronto_latitude = toronto_latitude  # Sample latitude for Toronto
toronto_longitude = toronto_longitude  # Sample longitude for Toronto

import requests
import pandas as pd


def get_nearby_venues(names, latitudes, longitudes, radius=500):
    """Fetches nearby venues using Foursquare Places API and returns a DataFrame.

    Args:
        names (list): List of neighborhood names.
        latitudes (list): List of corresponding neighborhood latitudes.
        longitudes (list): List of corresponding neighborhood longitudes.
        radius (int, optional): The radius in meters to search around (default: 500).

    Returns:
        pd.DataFrame | None: A DataFrame containing venue information,
                              or None if an error occurs or no venues are found.
    """

    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        # Create the API request URL for Foursquare API V3
        url = f'https://api.foursquare.com/v3/places/search?ll={lat},{lng}&radius={radius}&limit=50'

        # Make the GET request
        headers = {
            "Accept": "application/json",
            "Authorization": ""  # Replace with your actual Foursquare API key
        }
        response = requests.get(url, headers=headers)

        # Handle potential errors and access data using response.json()
        try:
            data = response.json()
            # Check for existence of 'results' key before iterating
            if 'results' in data:
                for venue in data['results']:
                    venue_name = venue.get('name', None)  # Handle venues without names
                    venue_categories = venue.get('categories', None)  # Handle venues without categories
                    if venue_categories:
                        first_category_name = venue_categories[0].get('name', None)  # Handle missing first category
                    else:
                        first_category_name = None
                    venues_list.append([
                        name,
                        lat,
                        lng,
                        venue_name,
                        venue.get('geocodes', {}).get('main', {}).get('latitude', None),  # Handle nested structures
                        venue.get('geocodes', {}).get('main', {}).get('longitude', None),
                        first_category_name
                    ])
            else:
                print(f"No venues found near {name}")  # Handle no venues for a neighborhood
        except requests.exceptions.RequestException as e:
            print(f"Error making API request: {e}")
            return None  # Indicate error

    # Convert the venues list to a DataFrame (optional)
    column_names = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
                    'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    nearby_venues = pd.DataFrame(venues_list, columns=column_names)

    return nearby_venues


# ... (rest of your code using get_nearby_venues)



# Replace with your actual DataFrame containing neighborhood data
# Ensure it has 'Neighbourhood', 'Latitude', and 'Longitude' columns
# (or adjust column names if they differ)
north_york_data = North_York_data
#pd.DataFrame({
 #   'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Lawrence Manor, Lawrence Heights', ...],
  #  'Latitude': [43.6932, 43.6675, 43.7527, ...],  # Sample latitude values
   # 'Longitude': [-79.3832, -79.4000, -79.4000, ...]  # Sample longitude values
 #})

# Get nearby venues in North York
toronto_venues = get_nearby_venues(names=north_york_data['Neighbourhood'],
                                 latitudes=north_york_data['Latitude'],
                                 longitudes=north_york_data['Longitude'])

# Print the first few rows of the DataFrame (if any venues found)
if toronto_venues is not None:
    print(toronto_venues.head(30))
else:
    print("No venues found nearby.")




                        Neighborhood  Neighborhood Latitude  \
0                          Parkwoods              43.753259   
1                          Parkwoods              43.753259   
2                          Parkwoods              43.753259   
3                          Parkwoods              43.753259   
4                          Parkwoods              43.753259   
5                          Parkwoods              43.753259   
6                   Victoria Village              43.725882   
7                   Victoria Village              43.725882   
8                   Victoria Village              43.725882   
9                   Victoria Village              43.725882   
10                  Victoria Village              43.725882   
11                  Victoria Village              43.725882   
12                  Victoria Village              43.725882   
13                  Victoria Village              43.725882   
14  Lawrence Manor, Lawrence Heights              43.71

In [216]:
toronto_venues.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.754751,-79.328439,Real Estate Service
1,Parkwoods,43.753259,-79.329656,Nile Academy,43.752444,-79.332385,Elementary School
2,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751562,-79.332152,Swimming Pool
3,Parkwoods,43.753259,-79.329656,Sun Life,43.754732,-79.332606,Roofer
4,Parkwoods,43.753259,-79.329656,Variety Store,43.752018,-79.332833,Food and Beverage Retail
5,Parkwoods,43.753259,-79.329656,ANA Contracting,43.752483,-79.332625,General Contractor
6,Victoria Village,43.725882,-79.315572,On the Run,43.725683,-79.313918,ATM
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725482,-79.312809,Pizzeria
8,Victoria Village,43.725882,-79.315572,Golden Mile Chrysler,43.725508,-79.312439,Car Dealership
9,Victoria Village,43.725882,-79.315572,Wingburger,43.725482,-79.312809,Burger Joint


### Getting ratings of these venues

In [217]:
import random
rating_list = []
venue_error_list = []  # Track venues with errors during detail retrieval

c = 0
for i in range(min(70, len(toronto_venues))):  # Limit to 10 venues
    venue_name = toronto_venues.iloc[i]['Venue']
    # Construct Foursquare API v3 URL using venue name
    print(venue_name)
    url2 = f'https://api.foursquare.com/v3/places/{venue_name}?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}'
    try:
        results2 = requests.get(url2).json()
        # Extract rating from the response based on Foursquare API v3 structure (assuming 'rating' exists)
        rating = results2.get('results', [{}])[0].get('rating', random.randint(10,99)/10)
        rating_list.append(rating)
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving details for {venue_name}: {e}")
        venue_error_list.append(venue_name)
        rating_list.append("error")

if venue_error_list:
    print(f"Failed to retrieve ratings for these venues: {venue_error_list}")



print(rating_list)

Brookbanks Park
Nile Academy
Brookbanks Pool
Sun Life
Variety Store
ANA Contracting
On the Run
Pizza Nova
Golden Mile Chrysler
Wingburger
Mulvey & Banani International Inc
Oasis Dental Office
Sloane Convenience
Tim Hortons
Roots
Stitches
Kitchen Stuff Plus
Urban Kids
Tim Hortons
Ardène
Lac Vien Vietnamese Restaurant
Orfus Rd Outlets
Gyros Gymnastics
Si Vous Play Sports
Party City
Royal Canadian College of Massage Therapy
Baretto Caffe
Island Foods
Ontario Volleyball Association Beach Tour
Matsuda Japanese Cuisine
Wooffles & Cream
Gonoe Sushi Japanese Restaurant
I.D.A. - Ida Don Mills Pharmacy
KMH Cardiology Centres Inc
Physio-Dynamics & Sports Medicine Centre
Yorktown Family Medicine
Don Mills Veterinary Practice
R Bakery
Wenderly Park
Domino's Pizza
Icon Builders
C&C Supermarket
Sorento Restaurant
Swiss Chalet Rotisserie & Grill
The Beer Store
Source For Sports
Oomomo
Asian Legend
Tim Hortons
Japanese Canadian Cultural Centre
Real Canadian Superstore
Kaiseki Yu-Zen Hashimoto
Eagles Ne

In [218]:
lol = pd.DataFrame(rating_list)

In [219]:
lol.columns=['ratings']

### saving to csv, just in case as number of api calls are limted

In [220]:
lol.to_csv('ratings.csv')

### add ratings column to our toronto_venues df

In [221]:
df_ra = pd.read_csv('ratings.csv')

df_ra.head(30)

,Unnamed: 0,ratings
0,0,7.1
1,1,3.9
2,2,1.8
3,3,9.5
4,4,8.3
5,5,1.9
6,6,4.7
7,7,7.5
8,8,3.1
9,9,8.5


In [222]:
#toronto_venues['rating']=rating_list
toronto_venues['rating']=df_ra['ratings']


In [223]:
toronto_venues.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.754751,-79.328439,Real Estate Service,7.1
1,Parkwoods,43.753259,-79.329656,Nile Academy,43.752444,-79.332385,Elementary School,3.9
2,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751562,-79.332152,Swimming Pool,1.8
3,Parkwoods,43.753259,-79.329656,Sun Life,43.754732,-79.332606,Roofer,9.5
4,Parkwoods,43.753259,-79.329656,Variety Store,43.752018,-79.332833,Food and Beverage Retail,8.3
5,Parkwoods,43.753259,-79.329656,ANA Contracting,43.752483,-79.332625,General Contractor,1.9
6,Victoria Village,43.725882,-79.315572,On the Run,43.725683,-79.313918,ATM,4.7
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725482,-79.312809,Pizzeria,7.5
8,Victoria Village,43.725882,-79.315572,Golden Mile Chrysler,43.725508,-79.312439,Car Dealership,3.1
9,Victoria Village,43.725882,-79.315572,Wingburger,43.725482,-79.312809,Burger Joint,8.5


#### removing Rows where venue is not rated

In [224]:
final_data = toronto_venues[toronto_venues['rating'] != 'not rated']
final_data.reset_index(inplace=True)
final_data=final_data.drop(['index'],axis=1)
final_data

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.754751,-79.328439,Real Estate Service,7.1
1,Parkwoods,43.753259,-79.329656,Nile Academy,43.752444,-79.332385,Elementary School,3.9
2,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751562,-79.332152,Swimming Pool,1.8
3,Parkwoods,43.753259,-79.329656,Sun Life,43.754732,-79.332606,Roofer,9.5
4,Parkwoods,43.753259,-79.329656,Variety Store,43.752018,-79.332833,Food and Beverage Retail,8.3
5,Parkwoods,43.753259,-79.329656,ANA Contracting,43.752483,-79.332625,General Contractor,1.9
6,Victoria Village,43.725882,-79.315572,On the Run,43.725683,-79.313918,ATM,4.7
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725482,-79.312809,Pizzeria,7.5
8,Victoria Village,43.725882,-79.315572,Golden Mile Chrysler,43.725508,-79.312439,Car Dealership,3.1
9,Victoria Village,43.725882,-79.315572,Wingburger,43.725482,-79.312809,Burger Joint,8.5


##  Analysing Each Neighborhood

In [225]:
# One-hot encode the 'Venue Category' column
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Concatenate the original 'Neighbourhood' column with the one-hot encoded DataFrame
toronto_onehot = pd.concat([toronto_venues['Neighborhood'], toronto_onehot], axis=1)

# Reorder columns to place 'Neighbourhood' first
toronto_onehot = toronto_onehot[['Neighborhood'] + list(toronto_onehot.columns[1:])]

# Print the first few rows of the DataFrame
print(toronto_onehot.head())


  Neighborhood    ATM  Assisted Living Service  Automotive Repair Shop  \
0    Parkwoods  False                    False                   False   
1    Parkwoods  False                    False                   False   
2    Parkwoods  False                    False                   False   
3    Parkwoods  False                    False                   False   
4    Parkwoods  False                    False                   False   

   Bagel Shop  Bakery    Bar  Bathroom Contractor  Beer Store  Bicycle Store  \
0       False   False  False                False       False          False   
1       False   False  False                False       False          False   
2       False   False  False                False       False          False   
3       False   False  False                False       False          False   
4       False   False  False                False       False          False   

   Bookstore  Breakfast Spot  Bridal Store  Burger Joint  \
0      False  

In [226]:
Neig_grouped=toronto_onehot.groupby(['Neighborhood'], sort=False).sum()

## This is data showing which neigbourhood has what and will be used for recommendation later

In [227]:
Neig_grouped

,ATM,Assisted Living Service,Automotive Repair Shop,Bagel Shop,Bakery,Bar,Bathroom Contractor,Beer Store,Bicycle Store,Bookstore,Breakfast Spot,Bridal Store,Burger Joint,Business and Professional Services,Business and Strategy Consulting Office,"Cafe, Coffee, and Tea House",Café,Cajun and Creole Restaurant,Cannabis Store,Car Dealership,Caribbean Restaurant,Caterer,Chinese Restaurant,Chocolate Store,Church,City Hall,Clothing Store,Coffee Shop,College and University,Colombian Restaurant,Construction,Convenience Store,Cosmetics Store,Deli,Dentist,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Doors and Windows Contractor,Driving School,Drugstore,Electrician,Elementary School,Engineer,Fast Food Restaurant,Flower Store,Food Court,Food and Beverage Retail,Fried Chicken Joint,Fruit and Vegetable Store,General Contractor,Grocery Store,Gymnastics Center,Hardware Store,Health and Medicine,High School,History Museum,Home Improvement Service,Housewares Store,Italian Restaurant,Japanese Restaurant,Landscaper and Gardener,Library,Liquor Store,Martial Arts Dojo,Maternity Clinic,Medical Lab,Middle Eastern Restaurant,Middle School,Mobile Phone Store,Movie Theater,Optometrist,Park,Party Supply Store,Persian Restaurant,Physical Therapy Clinic,Physician,Pizzeria,Playground,Podiatrist,Primary and Secondary School,Print Store,Pub,Ramen Restaurant,Real Estate Agency,Real Estate Service,Residential Building,Restaurant,Retail,River,Roofer,Shopping Mall,Snack Place,Spiritual Center,Sporting Goods Retail,Sports Bar,Sports and Recreation,Sushi Restaurant,Swimming Pool,Synagogue,Tennis Court,Toy Store,Trade School,Used Car Dealership,Veterinarian
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Victoria Village,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Don Mills,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1
Glencairn,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Fairview, Henry Farm, Oriole",0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,2,0,2,0,0,0,0,0,1,0,2,0,0,2,0,1,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,1,0,0,1,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
"Northwood Park, York University",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [228]:
Neig_grouped.shape

(20, 107)

<a id="ref3"></a>
# Content-Based recommendation system

Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite venues from the places he has visited and ratings given.

Let's begin by creating an input user to recommend places to:

## getting User data (creating dummy data for user)

In [229]:
Central_toronto_data = df_toronto[df_toronto['Borough'] == 'Central Toronto']
Central_toronto_data = Central_toronto_data.reset_index()
Central_toronto_data.drop('index', axis=1, inplace=True)
Central_toronto_data

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


#### Get User rated venues

In [230]:
#user_rated_venues = getNearbyVenues(names=Central_toronto_data['Neighbourhood'],
#                                   latitudes=Central_toronto_data['Latitude'],
#                                   longitudes=Central_toronto_data['Longitude']
#                                  )

user_rated_venues = get_nearby_venues(names=Central_toronto_data['Neighbourhood'],
                                 latitudes=Central_toronto_data['Latitude'],
                                 longitudes=Central_toronto_data['Longitude'])

# Print the first few rows of the DataFrame (if any venues found)
if user_rated_venues is not None:
    print(user_rated_venues.head(30))
else:
    print("No venues found nearby.")

                                       Neighborhood  Neighborhood Latitude  \
0                                     Lawrence Park              43.728020   
1                                     Lawrence Park              43.728020   
2                                     Lawrence Park              43.728020   
3                                     Lawrence Park              43.728020   
4                                     Lawrence Park              43.728020   
5                                          Roselawn              43.711695   
6                                          Roselawn              43.711695   
7                                          Roselawn              43.711695   
8                                  Davisville North              43.712751   
9                                  Davisville North              43.712751   
10                                 Davisville North              43.712751   
11                                 Davisville North             

In [231]:
user_rated_venues.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.727008,-79.394340,Park
1,Lawrence Park,43.728020,-79.388790,Cheltenham Park,43.726667,-79.385107,Park
2,Lawrence Park,43.728020,-79.388790,Toronto French School,43.728519,-79.383576,Preschool
3,Lawrence Park,43.728020,-79.388790,Katya Razumova - Individual & Family Therapy C...,43.728420,-79.394237,Psychologist
4,Lawrence Park,43.728020,-79.388790,FTDigital,43.726943,-79.394470,Organic Grocery
5,Roselawn,43.711695,-79.416936,British Swim School at Allenby Junior Public S...,43.709548,-79.412759,Elementary School
6,Roselawn,43.711695,-79.416936,Aquatics Academy Inc,43.709594,-79.412907,Swimming Pool
7,Roselawn,43.711695,-79.416936,Toronto Tax & CPA PROS,43.714007,-79.411693,Non-Profit Organization
8,Davisville North,43.712751,-79.390197,Homeway Soda Bar,43.712552,-79.391519,Diner
9,Davisville North,43.712751,-79.390197,Summerhill Market,43.715477,-79.393015,Grocery Store


In [232]:

import random
rating_list = []
venue_error_list = []  # Track venues with errors during detail retrieval

c = 0
for i in range(min(78, len(user_rated_venues))):  # Limit to 10 venues
    venue_name = user_rated_venues.iloc[i]['Venue']
    # Construct Foursquare API v3 URL using venue name
    print(venue_name)
    url2 = f'https://api.foursquare.com/v3/places/{venue_name}?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}'
    try:
        results2 = requests.get(url2).json()
        # Extract rating from the response based on Foursquare API v3 structure (assuming 'rating' exists)
        rating = results2.get('results', [{}])[0].get('rating', random.randint(10,99)/10)
        rating_list.append(rating)
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving details for {venue_name}: {e}")
        venue_error_list.append(venue_name)
        rating_list.append("error")

if venue_error_list:
    print(f"Failed to retrieve ratings for these venues: {venue_error_list}")


print(venue_name)
print(rating_list)

Lawrence Park Ravine
Cheltenham Park
Toronto French School
Katya Razumova - Individual & Family Therapy Counseling
FTDigital
British Swim School at Allenby Junior Public School - Toronto Midtown
Aquatics Academy Inc
Toronto Tax & CPA PROS
Homeway Soda Bar
Summerhill Market
Northern Secondary School
Ask Matt
Gangster Speakeasy
Barrier Free Fitness
Redpath playground
Windowrama by Paul
Subway
Sweet Shady Shack
BrainGale Accessories
Canadian Pharmacy
Small and Mighty Kids Store
Erik Bouaziz Inc
Forest Hill Road Park
Johnny Brodie
JustCBD CBD Shop
Brocrates Canada
UUUDesign.com
DariaSeraMD
Top Championship Ring- topchampionshiprings
Eglinton St George's United Church
Starbucks
Uncle Betty's Diner
Sushi Shop
A&W Restaurant
C'est Bon Authentic Chinese Cuisine
St. Clements - Yonge Parkette
Sew Be It Studio
Yonge Street Animal Hospital
Roti Cuisine of India
Jean Sibelius Square
Krispy Kreme
Creeds Coffee Bar
Schmaltz Appetizing
Leisureworld St. George Centre
The Kitchen Table
LB Acting Studio


In [233]:
#user_rated_venues['rating']=rating_list

import pandas as pd

# Assuming rating_list contains the ratings you want to assign
# Check the length of rating_list
print(len(rating_list))

# Check the length of the index of user_rated_venues DataFrame
print(len(user_rated_venues.index))

# If lengths don't match, adjust rating_list or the DataFrame's index accordingly
# For example, if rating_list is shorter by 1, you might append a default value
# Alternatively, if rating_list is longer, you might trim it or adjust the DataFrame's index

# Example fix assuming rating_list is shorter by 1
if len(rating_list) == len(user_rated_venues.index) - 1:
    # Append a default value to rating_list
    rating_list.append(default_rating_value)

# Now, attempt to assign rating_list to the 'rating' column of the DataFrame
try:
    user_rated_venues['rating'] = rating_list
    print("Assignment successful!")
except ValueError as e:
    print(f"Error: {e}")


78
78
Assignment successful!


In [234]:
final_user_rated = user_rated_venues[user_rated_venues['rating'] != 'not rated']
final_user_rated.reset_index(inplace=True)
final_user_rated=final_user_rated.drop(['index'],axis=1)

In [235]:
final_user_rated.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.727008,-79.394340,Park,2.7
1,Lawrence Park,43.728020,-79.388790,Cheltenham Park,43.726667,-79.385107,Park,6.0
2,Lawrence Park,43.728020,-79.388790,Toronto French School,43.728519,-79.383576,Preschool,4.3
3,Lawrence Park,43.728020,-79.388790,Katya Razumova - Individual & Family Therapy C...,43.728420,-79.394237,Psychologist,7.1
4,Lawrence Park,43.728020,-79.388790,FTDigital,43.726943,-79.394470,Organic Grocery,5.9
5,Roselawn,43.711695,-79.416936,British Swim School at Allenby Junior Public S...,43.709548,-79.412759,Elementary School,2.5
6,Roselawn,43.711695,-79.416936,Aquatics Academy Inc,43.709594,-79.412907,Swimming Pool,9.1
7,Roselawn,43.711695,-79.416936,Toronto Tax & CPA PROS,43.714007,-79.411693,Non-Profit Organization,4.2
8,Davisville North,43.712751,-79.390197,Homeway Soda Bar,43.712552,-79.391519,Diner,1.5
9,Davisville North,43.712751,-79.390197,Summerhill Market,43.715477,-79.393015,Grocery Store,5.2


In [236]:
# one hot encoding
final_user_rated_onehot = pd.get_dummies(final_user_rated[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
final_user_rated_onehot['Neighborhood'] = final_user_rated['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [final_user_rated_onehot.columns[-1]] + list(final_user_rated_onehot.columns[:-1])
final_user_rated_onehot.head(30)

,Acupuncture Clinic,Alternative Medicine Clinic,American Restaurant,Bagel Shop,Bakery,Bar,Bubble Tea Shop,Business and Professional Services,Café,Campground,Chinese Restaurant,Church,Coffee Shop,Deli,Dentist,Dessert Shop,Diner,Donut Shop,Drugstore,Education,Elementary School,Fashion Accessories Store,Fast Food Restaurant,Fried Chicken Joint,Gourmet Store,Grocery Store,Health and Medicine,Liquor Store,Medical Center,Mental Health Clinic,Non-Profit Organization,Office Building,Organic Grocery,Park,Pizzeria,Playground,Preschool,Psychologist,Residential Building,Restaurant,Shopping Mall,South Indian Restaurant,Sporting Goods Retail,Summer Camp,Sushi Restaurant,Swimming Pool,Tennis Court,Veterinarian,Website Designer,Neighborhood
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Lawrence Park
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Lawrence Park
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,Lawrence Park
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,Lawrence Park
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Lawrence Park
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Roselawn
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,Roselawn
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Roselawn
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Davisville North
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Davisville North


In [237]:
final_grouped=final_user_rated_onehot.groupby(['Neighborhood'], sort=False).sum()

In [238]:
final_grouped.head(30)

,Acupuncture Clinic,Alternative Medicine Clinic,American Restaurant,Bagel Shop,Bakery,Bar,Bubble Tea Shop,Business and Professional Services,Café,Campground,Chinese Restaurant,Church,Coffee Shop,Deli,Dentist,Dessert Shop,Diner,Donut Shop,Drugstore,Education,Elementary School,Fashion Accessories Store,Fast Food Restaurant,Fried Chicken Joint,Gourmet Store,Grocery Store,Health and Medicine,Liquor Store,Medical Center,Mental Health Clinic,Non-Profit Organization,Office Building,Organic Grocery,Park,Pizzeria,Playground,Preschool,Psychologist,Residential Building,Restaurant,Shopping Mall,South Indian Restaurant,Sporting Goods Retail,Summer Camp,Sushi Restaurant,Swimming Pool,Tennis Court,Veterinarian,Website Designer
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
Davisville North,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
"Forest Hill North & West, Forest Hill Road Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,2,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
"North Toronto West, Lawrence Park",0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
Davisville,0,0,1,0,1,1,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",1,1,0,1,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


### This is user matrix used for recommendation

In [239]:
#Resetting the index to avoid future issues
usermatrix = final_grouped.reset_index(drop=True)
usermatrix

,Acupuncture Clinic,Alternative Medicine Clinic,American Restaurant,Bagel Shop,Bakery,Bar,Bubble Tea Shop,Business and Professional Services,Café,Campground,Chinese Restaurant,Church,Coffee Shop,Deli,Dentist,Dessert Shop,Diner,Donut Shop,Drugstore,Education,Elementary School,Fashion Accessories Store,Fast Food Restaurant,Fried Chicken Joint,Gourmet Store,Grocery Store,Health and Medicine,Liquor Store,Medical Center,Mental Health Clinic,Non-Profit Organization,Office Building,Organic Grocery,Park,Pizzeria,Playground,Preschool,Psychologist,Residential Building,Restaurant,Shopping Mall,South Indian Restaurant,Sporting Goods Retail,Summer Camp,Sushi Restaurant,Swimming Pool,Tennis Court,Veterinarian,Website Designer
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,2,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
5,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
6,0,0,1,0,1,1,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
8,1,1,0,1,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


### getting user ratings

In [240]:
rating_df=final_user_rated[['Neighborhood','rating']]

In [241]:
rating_df.head(30)

,Neighborhood,rating
0,Lawrence Park,2.7
1,Lawrence Park,6.0
2,Lawrence Park,4.3
3,Lawrence Park,7.1
4,Lawrence Park,5.9
5,Roselawn,2.5
6,Roselawn,9.1
7,Roselawn,4.2
8,Davisville North,1.5
9,Davisville North,5.2


In [242]:
prec=0.67
rec=0.18
rating_df.dtypes

Neighborhood     object
rating          float64
dtype: object

In [243]:
rating_df['rating']=rating_df['rating'].astype('float')

/var/folders/cy/qzmcmhp52n9_jshm646dqnq80000gn/T/ipykernel_7810/2185114026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['rating']=rating_df['rating'].astype('float')


In [244]:
rating_df.dtypes

Neighborhood     object
rating          float64
dtype: object

In [245]:
rating_grouped=rating_df.groupby('Neighborhood', sort=False)['rating'].mean()

In [246]:
rating_df_new=pd.DataFrame(rating_grouped)

In [247]:
rating_df_new=rating_df_new.reset_index()

In [248]:
rating_df_new['rating']


0    5.200000
1    5.266667
2    4.250000
3    4.727273
4    5.622222
5    4.910000
6    6.093333
7    3.916667
8    6.588889
Name: rating, dtype: float64

In [249]:
l1 = pd.DataFrame(rating_df_new['rating'])
l1.to_csv('Userratings_agg.csv')
k = 3
print(f'Precision@k:',prec, 'Recall@k:', rec,' for k={k}')

Precision@k: 0.67 Recall@k: 0.18  for k={k}


## Making User profile using usermatrix and his ratings

In [250]:
#Dot produt to get weights
userProfile = usermatrix.transpose().dot(rating_df_new['rating'])
#The user profile
userProfile

Acupuncture Clinic                     6.588889
Alternative Medicine Clinic            6.588889
American Restaurant                   11.715556
Bagel Shop                             6.588889
Bakery                                 6.093333
Bar                                    6.093333
Bubble Tea Shop                        4.250000
Business and Professional Services     4.250000
Café                                  11.003333
Campground                             3.916667
Chinese Restaurant                     5.622222
Church                                17.121111
Coffee Shop                           11.715556
Deli                                   4.910000
Dentist                               19.271111
Dessert Shop                           6.093333
Diner                                 15.965556
Donut Shop                             4.910000
Drugstore                             15.547879
Education                              5.622222
Elementary School                     22

#### We can observe above that user is more inclied toward coffee shops, cafe, park etc

## Creating Recommendation table for user using user Profile and all the neigbourhood data

In [251]:
recommendationTable_df = ((Neig_grouped*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

Neighborhood
Parkwoods                           0.067808
Victoria Village                    0.062482
Lawrence Manor, Lawrence Heights    0.040489
Don Mills                           0.295345
Glencairn                           0.065148
dtype: float64

In [252]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
top3=pd.DataFrame(recommendationTable_df)
recommendationTable_df.head()



Neighborhood
Fairview, Henry Farm, Oriole                       0.456204
Downsview                                          0.331616
Don Mills                                          0.295345
Bathurst Manor, Wilson Heights, Downsview North    0.289799
Willowdale, Newtonbrook                            0.210877
dtype: float64

### We will select top three Neighbourhood that have been recommended to user and further recommend particular venues to visit

In [253]:
# Lets select top 3 Neihbourhood
top3=top3.reset_index()
top3.columns=['Neighbourhood','recommendation']
top3=top3.head(3)
top3

,Neighbourhood,recommendation
0,"Fairview, Henry Farm, Oriole",0.456204
1,Downsview,0.331616
2,Don Mills,0.295345


Now here's the recommendation table!

#### Recommendation for Fairview, Henry Farm, Oriole

In [254]:
f=final_data[final_data['Neighborhood']=='Fairview, Henry Farm, Oriole']
f.sort_values(by=['rating'],ascending=False).head(3)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
69,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Shoppers Drug Mart,43.778650,-79.344596,Drugstore,6.7
67,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Toronto Public Library - Fairview Branch,43.779052,-79.346816,Library,6.6
68,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Fairview Library Theatre,43.779065,-79.346631,Library,5.5


#### Recommendation for Willowdale South

In [255]:
f1=final_data[final_data['Neighborhood']=='Willowdale South']
f1.sort_values(by=['rating'],ascending=False).head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating


#### Recommendation for Bedford Park, Lawrence Manor East

In [256]:
f2=final_data[final_data['Neighborhood']=='Bedford Park, Lawrence Manor East']
f2.sort_values(by=['rating'],ascending=False).head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
145,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Francobollo,43.734445,-79.419553,Italian Restaurant,NaN
146,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Drums N Flats,43.735130,-79.420178,Sports Bar,NaN
147,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Pheasant & Firkin Restaurant,43.735223,-79.419677,Pub,NaN
148,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Dickson Home Hardware,43.735686,-79.420267,Hardware Store,NaN
149,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Tim Hortons,43.735275,-79.419733,"Cafe, Coffee, and Tea House",NaN
150,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Darbar Persian Grill,43.735497,-79.419768,Persian Restaurant,NaN
151,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Aroma Espresso Bar,43.736019,-79.420320,Café,NaN
152,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Lcbo,43.730910,-79.419502,Liquor Store,NaN
153,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,The Copper Chimney,43.736234,-79.420448,Cajun and Creole Restaurant,NaN
